This notebook requires:
* the North Sea test case from the [getm-setups repository](https://github.com/getm-model/setups)
* the [TPXO9 atlas](https://www.tpxo.net/global/tpxo9-atlas) to calculate tides at the open boundaries

In [ ]:
getm_setups_dir = '../../../getm-setups'
tpxo_root = '../../../igotm/data/TPXO9'

In [ ]:
import datetime
import os.path

import numpy
import xarray
import otps2

import pygetm
import pygetm.domain
import pygetm.input
import pygetm.legacy

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
domain = pygetm.legacy.domain_from_topo(os.path.join(getm_setups_dir, 'NorthSea/Topo/NS6nm.v01.nc'), nlev=1)
pygetm.legacy.load_bdyinfo(domain, os.path.join(getm_setups_dir, 'NorthSea/bdyinfo.dat'))
sim = pygetm.Simulation(domain, runtype=1, advection_scheme=1)

fig, ax = matplotlib.pyplot.subplots()
ax.pcolormesh(domain.T.loni, domain.T.lati, numpy.ma.array(domain.T.H, mask=domain.T.mask==0));

In [ ]:
ax.plot(domain.T.lon[domain.bdy_j, domain.bdy_i], domain.T.lat[domain.bdy_j, domain.bdy_i], 'ow', mec='k', ms=3);

In [ ]:
fig, ax = matplotlib.pyplot.subplots()

ax.pcolormesh(domain.T.loni, domain.T.lati, domain.T.mask);

In [ ]:
bdy_lon = domain.T.lon[domain.bdy_j, domain.bdy_i]
bdy_lat = domain.T.lat[domain.bdy_j, domain.bdy_i]

components = {}
for component in ('m2', 's2', 'n2', 'k2', 'k1', 'o1', 'p1', 'q1', 'm4', 'ms4', 'mn4', '2n2'):
    print('Computing %s on boundary...' % component)
    name = 'h_%s_tpxo9_atlas_30.nc' % component
    path = os.path.join(tpxo_root, name)
    with xarray.open_dataset(path) as ds:
        ds = ds.set_coords(('lat_z', 'lon_z'))
        re = ds['hRe']
        im = ds['hIm']
        components[component] = re.getm.interp(bdy_lon % 360., bdy_lat).values, im.getm.interp(bdy_lon % 360., bdy_lat).values

In [ ]:
ref_date = datetime.datetime(2000, 1, 1)
fig, ax = matplotlib.pyplot.subplots()
ax.pcolormesh(domain.T.loni, domain.T.lati, numpy.ma.array(domain.T.mask > 0, mask=domain.T.mask > 0))
h = otps2.predict_tide_2d(components, bdy_lat, ref_date, ntime=1, delta_time=0)[0, ...]
sc = ax.scatter(bdy_lon, bdy_lat, c=h, edgecolors='k', vmin=-2500, vmax=2500)
cb = fig.colorbar(sc)
cb.set_label('elevation (mm)')
title = ax.set_title(ref_date.strftime('%Y-%m-%d %H:%M'))

In [ ]:
for hour in range(24 * 365):
    date = ref_date + datetime.timedelta(seconds=hour*3600)
    h = otps2.predict_tide_2d(components, bdy_lat, date, ntime=1, delta_time=0)[0, ...]
    title.set_text(date.strftime('%Y-%m-%d %H:%M'))
    sc.set_array(h)
    fig.canvas.draw()